## Data Web Crawling

In [1]:
import pandas as pd
import numpy as np

import platform
import matplotlib.pyplot as plt

%matplotlib inline


In [2]:
from selenium import webdriver
import time 

In [3]:
driver = webdriver.Chrome('../../Driver/chromedriver')  #크롬드라이버 사용
driver.get('http://aws.seoul.go.kr/')

In [4]:
driver.find_element_by_xpath("""//*[@id="MENU_3"]""").click() 

In [5]:
driver.find_element_by_xpath("""//*[@id="MENU_S4"]""").click() 

In [6]:
# driver switch ->  http 이중구조, 내부 http로 전환
driver.switch_to.frame('MainFrame')


# 이중구조라 xpath가 듣지 않음, name 으로 태그 따오기
#station_list_raw = driver.find_element_by_xpath("""/*[name()='html']/*[name()='body']/*[name()='form']/*[name()='table']/*[name()='tbody']/*[name()='tr']
#/*[name()='td']/*[name()='table']/*[name()='tbody']/*[name()='tr']/*[name()='td']/*[name()='select']""")
station_list_raw = driver.find_element_by_name("AWS_ID")

In [7]:
station_list_raw
#/html/body/form/table/tbody/tr[2]/td[2]/table[1]/tbody/tr[1]/td[2]/select

<selenium.webdriver.remote.webelement.WebElement (session="a36230acf83742560a9f05720337a37e", element="0.9931237531915253-1")>

In [8]:
station_list = station_list_raw.find_elements_by_tag_name("option") # tag name 으로 찾기
station_names_values = [option.text for option in station_list]
station_names_values = station_names_values[1:]
station_names_values

['[서] 남산',
 '[서] 동대문',
 '[서] 성북',
 '[서] 도봉',
 '[서] 마포',
 '[서] 구로',
 '[서] 영등포',
 '[서] 서초',
 '[서] 강남',
 '[서] 서대문',
 '[서] 금천',
 '[서] 양천',
 '[서] 노원',
 '[서] 광진',
 '[서] 강북',
 '[서] 송파',
 '[서] 용산',
 '[서] 성동',
 '[서] 중랑',
 '[서] 중구',
 '[서] 종로',
 '[서] 은평',
 '[서] 동작',
 '[서] 관악',
 '[서] 강동',
 '[서] 강서',
 '[기] 서울',
 '[기] 강남',
 '[기] 서초',
 '[기] 강동',
 '[기] 송파',
 '[기] 강서',
 '[기] 양천',
 '[기] 도봉',
 '[기] 노원',
 '[기] 동대문',
 '[기] 중랑',
 '[기] 기상청',
 '[기] 마포',
 '[기] 서대문',
 '[기] 광진',
 '[기] 성북',
 '[기] 용산',
 '[기] 은평',
 '[기] 금천',
 '[기] 한강',
 '[기] 중구',
 '[기] 북악산',
 '[기] 성동',
 '[기] 구로',
 '[기] 강북',
 '[기] 관악',
 '[기] 영등포',
 '[기] 남현',
 '[기] 현충원']

In [9]:
station_names_values.index('[기] 서울')

26

In [10]:
station_names_values[:26]  # 기상청 자료는 쓰지않음, 서울시 관측소만

['[서] 남산',
 '[서] 동대문',
 '[서] 성북',
 '[서] 도봉',
 '[서] 마포',
 '[서] 구로',
 '[서] 영등포',
 '[서] 서초',
 '[서] 강남',
 '[서] 서대문',
 '[서] 금천',
 '[서] 양천',
 '[서] 노원',
 '[서] 광진',
 '[서] 강북',
 '[서] 송파',
 '[서] 용산',
 '[서] 성동',
 '[서] 중랑',
 '[서] 중구',
 '[서] 종로',
 '[서] 은평',
 '[서] 동작',
 '[서] 관악',
 '[서] 강동',
 '[서] 강서']

In [11]:
# Crawling data with Function   14년~ 17년

#다운로드 전에 드라이버의 다운로드 경로를 옮겨준다. Project폴더 생성 및 경로 변경
def date_download(area):
    import datetime
    import time
    from tqdm import tqdm_notebook
    import os
    from selenium import webdriver
    
    element = driver.find_element_by_name("AWS_ID")
    element.send_keys(station_names_values[area])
    start = datetime.datetime.strptime("2014-01-01", "%Y-%m-%d")
    end = datetime.datetime.strptime("2017-12-31", "%Y-%m-%d")
    date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days+1)] # 날짜 생성

    
    for n in tqdm_notebook(range(len(date_generated))):
    
        inputElement=driver.find_element_by_name('RptSDATE')  # name 으로 조회일자 접근
        inputElement.clear() # 조회일자 클리어
        inputElement.send_keys(date_generated[n].strftime('%Y-%m-%d')) # 날짜 넣기
    
        time.sleep(1)
    
        element_get_excel = driver.find_element_by_name('btnExcel').click()  #엑셀로 받기
    

In [12]:
# webdriver download path변경하면서 다운하려했지만 지원안함, driver열때마다 바꿔야함

In [ ]:
for i in range(len(station_names_values[:26])):
    date_download(i)

In [ ]:
driver.close()

## .xls 파일 -> .csv 로 변환해주어야 함
Project 폴더에 파일 모두 csv로 변환
VBA 코드 사용
```C
Sub chgXLS2CSV()
pathExcel = "a\"
pathCSV = "b\"
MyName = Dir(pathExcel, vbDirectory)
FullFileName = ""
Do While MyName <> ""
   If MyName <> "." And MyName <> ".." Then
      If (GetAttr(pathExcel & MyName) And vbDirectory) <> vbDirectory Then
          FullFileName = pathExcel & MyName
          SaveFileName = pathCSV & Left(MyName, Len(MyName) - 4) & ".csv"
          Workbooks.Open Filename:=FullFileName
          ActiveWorkbook.Worksheets(1).SaveAs Filename:=SaveFileName, FileFormat:=xlCSV
          ActiveWorkbook.Close False
      End If
    End If
    MyName = Dir
Loop
End Sub
```

In [66]:
#작업 디렉토리 변경
import os
os.chdir('C:\\Users\\mjsdh\\Downloads\\Project')

In [48]:
os.getcwd()

'C:\\Users\\mjsdh\\Downloads\\Project'

In [51]:
#os 로 폴더 생성, 다운로드 데이터 정리
for i in range(len(station_names_values[:26])):
    os.mkdir(station_names_values[i])            # os.renames 쓴다면 굳이 안해도됨

In [59]:
# 폴더 정리
from glob import glob

for j in range(0,26):
    station=[]
    station=glob('*({0})*'.format(j))
    for i in range(len(young)):
        os.renames(station[i], 'C:/Users/mjsdh/Downloads/Project/'+station_names_values[j]+'/'+station[i]) 

In [64]:
 # 남산 따로 정리
print(station_names_values[0])
station=glob('*.xls')
for i in range(len(station)):
    os.renames(station[i], 'C:/Users/mjsdh/Downloads/Project/'+station_names_values[0]+'/'+station[i])

[서] 남산
